# Drive mount

In [1]:
#Colab에 파일 업로드 시 직접 업로드보다는 구글 드라이브 연동을 추천합니다.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Import

In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install datasets
!pip install transformers[torch] -U

In [3]:
import os
import json
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from tqdm import tqdm

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)
from datasets import load_dataset, Dataset, DatasetDict
from accelerate import Accelerator


# 데이터를 Colab 서버 내에 세팅

In [4]:
# open.zip 파일을 업로드한 경로를 확인하여 적용
!unzip  "/content/drive/MyDrive/2024인하대/open.zip"

Archive:  /content/drive/MyDrive/2024인하대/open.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Load Validation / Test dataset

In [5]:
file_path = '/content/train.csv'
train_data = pd.read_csv(file_path)

train_data = train_data.sample(frac=1).reset_index(drop=True)

val_data=train_data[:10]
train_data = train_data[10:]

val_label_df = val_data[['question', 'answer']]

train_data.head(2)

,id,context,question,answer
10,TRAIN_32092,한섬이 여성복 브랜드 ‘타임’의 BI(Brand Identity)를 28년 만에 바...,타임 로고와 슬로건에 변화를 줄 계획인 곳은 어떤 의류 업체야,한섬
11,TRAIN_33192,현대중공업그룹 조선계열사인 현대중공업은 지난 19일 조선업계 최초로 NICE신용평가...,현대중공업이 발행할 채권은 어디에 활용될 예정이야,친환경 선박 건조 및 기술개발


# Model Load

In [6]:
# llama-2-ko-7b 모델 로드
model_id = "beomi/llama-2-ko-7b"
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            torch_dtype="auto", load_in_4bit=True) # 한정된 메모리에서 사용하기 위해, 4비트 정밀도로 로드

tokenizer = AutoTokenizer.from_pretrained(model_id) # 본 모델의 토크나이저 로드
tokenizer.use_default_system_prompt = False  # 기본 시스템 프롬프트를 사용하지 않도록 설정

accelerator = Accelerator()

model, tokenizer = accelerator.prepare(model, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Define F1 score

In [7]:
def normalize_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        '''연속된 공백일 경우 하나의 공백으로 대체'''
        return ' '.join(text.split())

    def remove_punc(text):
        '''구두점 제거'''
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        '''소문자 전환'''
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()

    # 문자 단위로 f1-score를 계산 합니다.
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)

    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)

    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def evaluate(ground_truth_df, predictions_df):
    predictions = dict(zip(predictions_df['question'], predictions_df['answer']))
    f1 = exact_match = total = 0

    for index, row in ground_truth_df.iterrows():
        question_text = row['question']
        ground_truths = row['answer']
        total += 1
        if question_text not in predictions:
            continue
        prediction = predictions[question_text]
        f1 = f1 + f1_score(prediction, ground_truths)

    f1 = 100.0 * f1 / total
    return {'f1': f1}


# Validation
- 로드한 모델에 대해 간단한 프롬프트를 입력하여 성능 검증

In [8]:
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [9]:
# 모델로 추론 후, 전처리를 수행한 뒤, 완성된 정답으로 반환합니다.
def generate_response(question_prompt):
    # 생성할 최대 토큰 수와, 답변 생성 수, 패딩 토큰의 idx를 지정하여 모델 파이프 라인을 설정하고, 답변을 생성합니다.
    response = qa_pipeline(question_prompt, max_new_tokens=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)[0]['generated_text']
    if "Answer:" in response:
            # Answer: 이후에 생성된 토큰 들만을 답변으로 사용합니다.
            response = response.split("Answer:", 1)[1][:20]

            # 토큰 반복 생성 및 노이즈 토큰 관련 처리
            if "Que" in response:
                response = response.split("Que", 1)[0]
            if "⊙" in response:
                response = response.split("⊙", 1)[0]
            if "Con" in response:
                response = response.split("Con", 1)[0]
    return response

In [10]:
#Google Colab T4 GPU 기준 10개 샘플 답안 생성에 1분 소요
predict_dict = {}
count = 0

for index, row in val_data.iterrows():
    try:
        context = row['context']
        question = row['question']


        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            predict_dict[question] = answer
        else:
            predict_dict[question] = 'Invalid question or context'

        print("Answer for question:", question, ":", predict_dict[question])
        count += 1
        print("Processed count:", count)
    except Exception as e:
        print(f"Error processing question {e}")



Answer for question: 조폐공사가 장애인들의 창작물을 반영해서 만든 것은 뭐야 :  아트메달이야.​​​​​​​​​​​​
Processed count: 1
Answer for question: 양 기관의 협력 기간은 얼마 동안이어야 해 :  5년 
Processed count: 2
Answer for question: 뉴 포드 익스페디션 자동차의 연비가 얼마야 :  L당 7.4km​​​​​​​​​​​
Processed count: 3
Answer for question: 임시 선박 투입 수는 매월 몇 척 이상인지 :  매월 2척 이상​​​​​​​​​​​
Processed count: 4
Answer for question: 청주에서 외국인 노동자를 쓸 수 있도록 허가할 예정인 사업소의 수는 얼마야 :  86개 사업소 
Processed count: 5
Answer for question: 찰스 스콧상이 만들어진 때는 언제야 :  1995년 ​​​​​​​​​​​​​
Processed count: 6
Answer for question: 그루스터디카페 창업 설명회에 참여하려면 어디에 접수하면 돼 :  그루스터디카페 홈페이지에 접수하면 
Processed count: 7
Answer for question: 바이드노믹스의 경기부양 시책 중 가장 중심이 되는 부분이 뭐야 :  현금보조금 지급이야. 
Processed count: 8
Answer for question: 시흥시 중소기업협동조합은 다방면에 걸쳐 공동사업을 추진해 어디에 이바지하고 있어 :  중소기업협동조합은 중소기업의 공동이
Processed count: 9
Answer for question: 팬데믹 때문에 타격을 입은 경제를 되살리는 데 중소기업이 앞장서겠다고 말한 사람이 누구야 :  박승균 중소기업중앙회 강원중소기업회
Processed count: 10


In [11]:
val_inf_df = pd.DataFrame(list(predict_dict.items()), columns=['question', 'answer'])
val_inf_df.head()

,question,answer
0,조폐공사가 장애인들의 창작물을 반영해서 만든 것은 뭐야,아트메달이야.​​​​​​​​​​​​
1,양 기관의 협력 기간은 얼마 동안이어야 해,5년
2,뉴 포드 익스페디션 자동차의 연비가 얼마야,L당 7.4km​​​​​​​​​​​


In [12]:
# f1-score 를 출력합니다.
results = evaluate( val_inf_df, val_label_df)
print(results)

{'f1': 41.090895882657904}


# Inference

In [13]:
file_path = '/content/test.csv'
test_data = pd.read_csv(file_path)

In [ ]:
# 모델 추론
submission_dict = {}

for index, row in test_data.iterrows():
    try:
        context = row['context']
        question = row['question']
        id = row['id']

        if context is not None and question is not None:
            question_prompt = f"너는 주어진 Context를 토대로 Question에 답하는 챗봇이야. \
                                Question에 대한 답변만 가급적 한 단어로 최대한 간결하게 답변하도록 해. \
                                Context: {context} Question: {question}\n Answer:"

            answer = generate_response(question_prompt)
            submission_dict[id] = answer
        else:
            submission_dict[id] = 'Invalid question or context'

    except Exception as e:
        print(f"Error processing question {e}")

# Submission

In [ ]:
# 제출
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])
df.to_csv( '/content/submission.csv', index=False)